In [1]:
from datasets import load_dataset
from entropyrank import EntropyRank
import rake_nltk
import yake
from keybert import KeyBERT
from TextRank import TextRank


c:\Users\tsale\OneDrive\Desktop\CS Masters Degree\EntropyRank\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def generate_stop_words_from_files(stop_words_files) -> set[str]:
        stop_words_from_files = set()
        for file_path in stop_words_files:
            # get current directory
            # current_dir = os.path.dirname(os.path.realpath(__file__))
            # file_path = os.path.join(current_dir, file_path)
            stop_words_from_files.update(
                EntropyRank._load_stop_words_from_file(file_path)
            )

        return stop_words_from_files

stop_words_files = [
            "stopwords/SmartStoplist.txt",
        ]

stop_words_from_files = generate_stop_words_from_files(stop_words_files)

In [5]:

entropy_extractor = EntropyRank()
rake_extractor = rake_nltk.Rake(include_repeated_phrases=False, stopwords=stop_words_from_files, max_length=5)
yake_extractor = yake.KeywordExtractor()
vectorizer=entropy_extractor.key_phrase_vectorizer
kw_model = KeyBERT()
text_rank_extractor = TextRank()


In [7]:
#  long document english scientific articles
semeval_2010 = load_dataset("midas/semeval2010", "generation")
# abstracts of english scientific articles
semeval_2017 = load_dataset("midas/semeval2017", "generation")
# abstracts of English scientific papers
inspec = load_dataset("midas/inspec", "generation")





Found cached dataset semeval2010 (C:/Users/tsale/.cache/huggingface/datasets/midas___semeval2010/generation/0.0.1/557ceb28d01a0c1a7d1f26fbfd46e2c6285979264b3880b40d2b2234dac51443)
100%|██████████| 2/2 [00:00<00:00, 76.92it/s]
Found cached dataset semeval2017 (C:/Users/tsale/.cache/huggingface/datasets/midas___semeval2017/generation/0.0.1/0871215fa4f7da4155260640dfcb03a62ce17eb9f459377232459bfc1b17ab94)
100%|██████████| 3/3 [00:00<00:00, 99.99it/s]
Found cached dataset inspec (C:/Users/tsale/.cache/huggingface/datasets/midas___inspec/generation/0.0.1/debd18641afb7048a36cee2b7bb8dfbf2cd1a68899118653a42fd760cf84284e)
100%|██████████| 3/3 [00:00<00:00, 90.86it/s]


In [8]:
from dataset_utils import prepare_ds_for_eval

inspec_docs, inspec_phrases = prepare_ds_for_eval(inspec)
semeval_2010_docs, semeval_2010_phrases = prepare_ds_for_eval(semeval_2010)
semeval_2017_docs, semeval_2017_phrases = prepare_ds_for_eval(semeval_2017)

In [10]:
# print len of each dataset
print(f"inspec: {len(inspec_docs)}")
print(f"semeval_2010: {len(semeval_2010_docs)}")
print(f"semeval_2017: {len(semeval_2017_docs)}")


inspec: 1984
semeval_2010: 244
semeval_2017: 500


### Prepare methods

In [11]:
from entropyrank import PartitionMethod


def rake_extraction(doc):
    rake_extractor.extract_keywords_from_text(doc)
    return rake_extractor.get_ranked_phrases()

text_rank_extraction = lambda doc: text_rank_extractor.extract_key_phrases(doc)
yake_extraction = lambda doc: [phrase[0] for phrase in yake_extractor.extract_keywords(doc)]
pattern_rank_extraction = lambda doc: [keyword[0] for keyword in kw_model.extract_keywords(doc, vectorizer=vectorizer, top_n=20)]
entropy_extraction_noun_phrases = lambda doc: [result[0] for result in entropy_extractor.extract_key_phrases(doc, number_of_key_phrases=20, partition_method=PartitionMethod.NOUN_PHRASES)]
entropy_extraction_stop_words = lambda doc: [result[0] for result in entropy_extractor.extract_key_phrases(doc, number_of_key_phrases=20, partition_method=PartitionMethod.STOP_WORDS)]

### Eval

In [8]:
from metrics import evaluate_metrics

k_values = [5, 10, 15]
methods = {
    "entropy_noun_phrases": entropy_extraction_noun_phrases,
    "pattern_rank": pattern_rank_extraction,
    "rake": rake_extraction,
    "yake": yake_extraction,
    "text_rank": text_rank_extraction,
}
data_sets = {
    "inspec": (inspec_docs, inspec_phrases),
    "semeval_2010": (semeval_2010_docs, semeval_2010_phrases),
    "semeval_2017": (semeval_2017_docs, semeval_2017_phrases),
}

results = {}
use_partial_match_values = [False]

for data_set_name, (docs, phrases) in data_sets.items():
    print(f"Dataset: {data_set_name}")
    for method_name, method in methods.items():
        print(f"Method: {method_name}")
        results_dict = evaluate_metrics(
            eval_set=docs,
            gt=phrases,
            extraction_callback=method,
            k_values=k_values,
            use_partial_match_values=use_partial_match_values,
        )
        if data_set_name not in results:
            results[data_set_name] = {}
        results[data_set_name][method_name] = results_dict
        print(results_dict)

Dataset: inspec
Method: entropy_noun_phrases


100%|██████████| 1984/1984 [04:28<00:00,  7.40it/s]


{'precision@5': 0.32207661290322476, 'recall@5': 0.2917951946309215, 'f1@5': 0.282588579964806, 'rouge1@5': 0.43802020312142, 'precision@10': 0.27469758064516214, 'recall@10': 0.4711736451621476, 'f1@10': 0.3238939559976106, 'rouge1@10': 0.4815145831692065, 'precision@15': 0.23195564516128572, 'recall@15': 0.5722799050474829, 'f1@15': 0.3110647081700049, 'rouge1@15': 0.4726735201362184}
Method: pattern_rank


100%|██████████| 1984/1984 [01:15<00:00, 26.44it/s]


{'precision@5': 0.3290322580645155, 'recall@5': 0.3098948867912688, 'f1@5': 0.29424339276976275, 'rouge1@5': 0.44506643006401275, 'precision@10': 0.28497983870967786, 'recall@10': 0.49703656499819426, 'f1@10': 0.33854773241686786, 'rouge1@10': 0.48709081014564726, 'precision@15': 0.24193548387096275, 'recall@15': 0.5995255287034565, 'f1@15': 0.3251405905243928, 'rouge1@15': 0.4855744047468491}
Method: rake


...

{'precision@5': 0.18000000000000071, 'recall@5': 0.08519146830157198, 'f1@5': 0.11121170371711316, 'rouge1@5': 0.2501469594046036, 'precision@10': 0.20639999999999975, 'recall@10': 0.18780819906125698, 'f1@10': 0.18892578559487702, 'rouge1@10': 0.39533065982245996, 'precision@15': 0.20773333333333402, 'recall@15': 0.2750547566147467, 'f1@15': 0.22767056647577438, 'rouge1@15': 0.45265431577832815}
Method: yake


100%|██████████| 500/500 [00:29<00:00, 16.98it/s]


{'precision@5': 0.18160000000000087, 'recall@5': 0.08201343890974662, 'f1@5': 0.10842020775671737, 'rouge1@5': 0.1985434971392111, 'precision@10': 0.17859999999999987, 'recall@10': 0.16117237981636137, 'f1@10': 0.1616673064184049, 'rouge1@10': 0.2858442033078512, 'precision@15': 0.16493333333333407, 'recall@15': 0.21881030055174422, 'f1@15': 0.18002258608483637, 'rouge1@15': 0.32666843407139423}
Method: text_rank


100%|██████████| 500/500 [00:12<00:00, 41.07it/s]


{'precision@5': 0.2568000000000009, 'recall@5': 0.11728231327593518, 'f1@5': 0.15481637994614694, 'rouge1@5': 0.25598820532240896, 'precision@10': 0.24199999999999983, 'recall@10': 0.21587648802221274, 'f1@10': 0.2187763837060686, 'rouge1@10': 0.37591974669721784, 'precision@15': 0.22386666666666738, 'recall@15': 0.2954140192977191, 'f1@15': 0.24418819826911542, 'rouge1@15': 0.4357497872012964}


In [9]:
print(results)

{'inspec': {'entropy_noun_phrases': {'precision@5': 0.32207661290322476, 'recall@5': 0.2917951946309215, 'f1@5': 0.282588579964806, 'rouge1@5': 0.43802020312142, 'precision@10': 0.27469758064516214, 'recall@10': 0.4711736451621476, 'f1@10': 0.3238939559976106, 'rouge1@10': 0.4815145831692065, 'precision@15': 0.23195564516128572, 'recall@15': 0.5722799050474829, 'f1@15': 0.3110647081700049, 'rouge1@15': 0.4726735201362184}, 'pattern_rank': {'precision@5': 0.3290322580645155, 'recall@5': 0.3098948867912688, 'f1@5': 0.29424339276976275, 'rouge1@5': 0.44506643006401275, 'precision@10': 0.28497983870967786, 'recall@10': 0.49703656499819426, 'f1@10': 0.33854773241686786, 'rouge1@10': 0.48709081014564726, 'precision@15': 0.24193548387096275, 'recall@15': 0.5995255287034565, 'f1@15': 0.3251405905243928, 'rouge1@15': 0.4855744047468491}, 'rake': {'precision@5': 0.21340725806451283, 'recall@5': 0.20599633267881282, 'f1@5': 0.19316944816351256, 'rouge1@5': 0.37388613892497957, 'precision@10': 0.2

In [10]:
# save results into a json file
import json
with open("results.json", "w") as f:
    json.dump(results, f, indent=4)

In [15]:
# iterate over the numerical values, multiply them by 100 and round them to 2 decimal places
for data_set_name, (docs, phrases) in data_sets.items():
    for method_name, method in methods.items():
        for key, value in results[data_set_name][method_name].items():
            if isinstance(value, float):
                results[data_set_name][method_name][key] = round(value * 100, 2)
        

In [ ]:
# save into final rounded results jsi file

with open("results_rounded.json", "w") as f:
    json.dump(results, f, indent=4)

In [21]:
from metrics import _lc_gt_and_results


def compute_average_intersection_over_union_at_k(
    results: list[list[str]], ground_truth: list[list[str]], k: int
) -> float:
    """Compute the average intersection over union at k for a list of results and ground truth."""
    return sum(
        compute_intersection_over_union_at_k(result, gt, k)
        for result, gt in zip(results, ground_truth)
    ) / len(results)


def compute_intersection_over_union_at_k(
    results: list[str], ground_truth: list[str], k: int
) -> float:
    """Compute the intersection over union at k for a list of results and ground truth."""
    # lower case all the results and ground truth
    results, ground_truth = _lc_gt_and_results(results, ground_truth)
    return len(set(results[:k]).intersection(set(ground_truth))) / len(
        set(results[:k]).union(set(ground_truth))
    )

In [26]:
# run compute_intersection_over_union_at_k at k = 15 for  entropy_extraction_noun_phrases, pattern_rank_extraction, on inspec
from tqdm import tqdm

k = [5, 10, 15]

entropy_np_results = []
pattern_rank_results = []
for eval_case in tqdm(inspec_docs):
        entropy_np_results.append(entropy_extraction_noun_phrases(eval_case))
        pattern_rank_results.append(pattern_rank_extraction(eval_case))

iou_at_k = {}
for k in k:
    iou_at_k[k] = compute_average_intersection_over_union_at_k(entropy_np_results, inspec_phrases, k)




    

100%|██████████| 1984/1984 [08:00<00:00,  4.13it/s]


In [27]:
print("Intersection over union at k = 15 for entropy_extraction_noun_phrases and pattern_rank_extraction on inspec: ", iou_at_k)


Intersection over union at k = 15 for entropy_extraction_noun_phrases and pattern_rank_extraction on inspec:  {5: 0.18091734739165935, 10: 0.21027611840149796, 15: 0.20428895484440596}
